In [1]:
import os
gpu_id = 1
print("Initlizing the process: %d with GPU: %d"%(os.getpid(),gpu_id))

# Configure which GPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_id}"

import numpy as np
import sionna
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray

scene = load_scene(sionna.rt.scene.munich)
scene.tx_array = PlanarArray(num_rows=1, 
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

tx = Transmitter(name="tx",
                 position=[8.5,21,27])

scene.add(tx)

# Compute coverage map
cm = scene.coverage_map(cm_cell_size=[1,1],
                        cm_center=[45,90,1.5],
                        cm_orientation=[0.,0.,0.],
                        cm_size=[10, 10],
                        num_samples=20e6)

# Place receivers at the center of all coverage map tiles
for i, pos in enumerate(np.reshape(cm.cell_centers, [-1,3])):
    scene.add(Receiver(name=f"rx-{i}",
                       position=pos,
                       orientation=[0,0,0]))

# Compute paths
paths = scene.compute_paths()

# Compute coverage map from paths
a, _ = paths.cir()
cm_theo = np.sum(np.abs(np.squeeze(a))**2, -1)
n = int(np.sqrt(cm_theo.shape[0]))
cm_theo = np.reshape(cm_theo, [n,n])

# Compute difference
nmse = np.mean(np.abs(cm_theo-cm[0])**2) / np.mean(np.abs(cm_theo)**2)
nmse_db = 10*np.log10(nmse)
print("NMSE (dB): ", nmse_db)

# Visualize coverage map in preview
scene.preview(coverage_map=cm)

Initlizing the process: 3283503 with GPU: 1


2023-07-14 13:21:32.381902: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-14 13:21:32.422118: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 13:21:33.023536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-14 13:21:34.783187: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38162 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:ca:00.0, compute capab

InternalError: {{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} Attempting to perform BLAS operation using StreamExecutor without BLAS support [Op:BatchMatMulV2] name: 